# Digging Into Deep Time and Deep Cover

<a href="https://doi.org/10.5281/zenodo.3875779"><img src="https://zenodo.org/badge/DOI/10.5281/zenodo.3875779.svg" align="right" alt="doi: 10.5281/zenodo.3875779" style="padding: 0px 10px 10px 0px"></a>
<a href="https://github.com/morganjwilliams/gs2020-diggingdeeper/blob/master/LICENSE"><img src="https://img.shields.io/badge/License-MIT-blue.svg" align="right" alt="License: MIT" style="padding: 0px 10px 10px 0px"></a>




<span id='authors'><b>Morgan Williams <a class="fa fa-twitter" aria-hidden="true" href="https://twitter.com/metasomite" title="@metasomite"></a></b>, Jens Klump, Steve Barnes and Fang Huang; </span>
<span id='affiliation'><em>CSIRO Mineral Resources</em></span>


### Contents

| [**Abstract**](./00_overview.ipynb) | **Introduction**                                                    | [**Examples**](./00_overview.ipynb#Examples)            | [**Tools**](./00_overview.ipynb#Tools) | [**Insights**](./00_overview.ipynb#Insights) |
|:-----|:-----|:-----|:-----|:-----|
|  | [Minerals Exploration](./00_overview.ipynb#An-Evolving-Role-of-Geochemistry-in-Mineral-Exploration)  | [Classification](./011_classification.ipynb) |  |  |
|  | [Data Driven Geochem](./00_overview.ipynb#Data-Driven-Geochemistry) | [Data Visualization](./012_datavis.ipynb) |  |  |

## Data Exploration

While there are many avenues to improve the robustness of data analysis in research, there are also opportunities to expand how we explore our datasets, potentially allowing us to find 'latent' features faster, or provide additional perspectives and context.

* Adding context to our data - what does it mean in the 'scheme of things'
* Making the most of the dimensions we have, but acknowledging that we have limited ability to understand complex data - dimensional reduction, smart visualisations (e.g. distance vs area vs volume).

**Key Messages**:

### Visualizing Uncertainty

While some classification problems can be adapted to versions of binary classifications, most involve multiple classes. In the case of probabilistic classification, model outputs are typically a multidimensional array of multiclass probability estimates, where the relative uncertainty of classification is more difficult to ascertain, and harder again to visualize for a larger group of samples. One measure which is related to the uncertainty of classification is the [information entropy](https://en.wikipedia.org/wiki/Entropy_(information_theory)), which effectively measures the distribution of probability among a number of potential classes:

\\[S = - \sum{P_i \cdot log(P_i)} \\]

Where samples are predicted to belong to one of mutliple classes with high relative probability, entropy is low. Where a sample is predicted to belong to mutliple classes with more or less equal probability, entropy is high. This is a particularly useful measure for quick visualisations of classification outputs, where the entropy is rescaled to \\([0,1]\\) to give a relative opacity.

In [ ]:
import numpy as np
import sklearn.datasets
import matplotlib.pyplot as plt
from pyrolite.util.skl.pipeline import SVC_pipeline
from pyrolite.util.skl.vis import alphas_from_multiclass_prob
from pyrolite.util.plot import DEFAULT_DISC_COLORMAP

np.random.seed(82)

In [ ]:
wine = sklearn.datasets.load_wine()
data, target = wine["data"], wine["target"]
svc = SVC_pipeline(probability=True)
gs = svc.fit(data, target)

In [ ]:
from scipy.stats import entropy
from pyrolite.util.plot.axes import share_axes
fig, ax = plt.subplots(3, 4, figsize=(8,7))
ax=ax.flat
share_axes(ax[:-1], which='xy')

cm = plt.cm.Greens

ax[0].set_ylim(0, 1)
n = 11
dt = gs.predict_proba(data)[::12][:n]
dtalpha = alphas_from_multiclass_prob(dt, method='entropy')
for ix in range(n):
    ax[ix].axhline(1/3, color='k', ls='--', lw=0.5)
    ax[ix].bar(['A', 'B', 'C'], dt[ix], alpha=dtalpha[ix], facecolor='purple')
    ax[ix].set_ylabel('Sample {}'.format(ix+1))
    ax[ix].annotate("S: {:.2f}".format(entropy(dt[ix]))+"\n"+r"$\alpha$: {:.2f}".format(dtalpha[ix]), 
                    xy=(0.95, 0.95), 
                    xycoords='axes fraction',
                    ha='right', 
                    va='top')

ax[-1].scatter([entropy(dt[ix]) for ix in range(n)], [dtalpha[ix]for ix in range(n)], color='k')
ax[-1].set_xlabel('Entropy')
ax[-1].set_ylabel('alpha')
plt.tight_layout()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(8, 4))
x, y = data[:, f0],data[:, f1],

classes = gs.predict(data)
c = DEFAULT_DISC_COLORMAP(classes)

ax[0].scatter(x, y, c=c, s=50)

c[:, -1] = alphas_from_multiclass_prob(gs.predict_proba(data), method='entropy')
ax[1].scatter(x, y, c=c, s=50);

### Applying this to Geochemistry

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

import pyrolite.comp
import pyrolite.geochem
from pyrolite.util.units import scale
from pyrolite.util.skl.pipeline import SVC_pipeline
from pyrolite.util.skl.vis import plot_mapping
from pyrolite.util.plot import DEFAULT_DISC_COLORMAP
from pyrolite.util.plot.legend import proxy_line

df = pd.read_csv('https://storage.googleapis.com/aegc2019/ueki2018.csv')

df_chem = df.pyrochem.oxides.join(
    df.pyrochem.elements * scale('ppm', 'wt%')
).pyrocomp.ILR().join(
    df[[ 'Sr87Sr86', 'Nd143Nd144', 'Pb206Pb204', 'Pb207Pb204', 'Pb208Pb204']]
)

In [ ]:
classes = {c: ix for ix, c in enumerate(df.Class.unique())}
X, y = df_chem, df["Class"]
y = y.apply(lambda x: classes[x]) # turn these into integers for the colormap
svc = SVC_pipeline(probability=True, scaler=StandardScaler(), kernel='rbf')
gs = svc.fit(X, y)

In [ ]:
fig, ax = plt.subplots(1, 2, sharex=True, sharey=True, figsize=(8, 4))

xsample = X.sample(500)
s=40

a, tfm, mapped = plot_mapping(
    xsample, 
    gs.best_estimator_, 
    ax=ax[1], 
    s=s,
    init="pca"
)
ax[0].scatter(*mapped.T, c=DEFAULT_DISC_COLORMAP(gs.predict(xsample)), s=s)

ax[0].set_xticks([])
ax[0].set_yticks([])

ax[1].legend([proxy_line(c=DEFAULT_DISC_COLORMAP(ix), ls='none', marker='o') for c, ix in classes.items()], 
             [c for c in classes.keys()], 
             bbox_to_anchor=(1.05, 1.0), 
             loc='upper left', 
             frameon=False, 
             fontsize=12);

### Dimensional Reduction for Visualization

UMAP Uniform Manifold Approximation and Projection (UMAP) is a dimension reduction technique which can be used in a very similar way to other manifold methods within scikit-learn ([McInnes2018]), with the added flexibility of being able transform new/novel data. This is particularly useful for visualizing training and testing data together, but can also be applied to map reference compositions and boundaries into the projected space. UMAP can also be used for semi-supervised and supervised dimensional reduction and metric learning, but this is beyond the scope of these notebooks. The technique is based on a specific set of assumptions which may be violated in practice, but it remains effective for the uses described here.

[McInnes2018]: https://doi.org/10.21105/joss.00861 "McInnes, L., Healy, J., Saul, N., Grossberger, L., 2018. UMAP: uniform manifold approximation and projection. The Journal of Open Source Software 3, 861."

In [ ]:
import umap.umap_ as umap  # required in case of umap install errors

reducer = umap.UMAP(n_neighbors=100, min_dist=0.01)
embedding = reducer.fit_transform(df_chem)

In [ ]:
fig, ax = plt.subplots(1, figsize=(5,5))

ax.scatter(*embedding.T, 
           c=DEFAULT_DISC_COLORMAP(gs.predict(df_chem)),
           s=s)
ax.set_xticks([])
ax.set_yticks([])

ax.legend([proxy_line(c=DEFAULT_DISC_COLORMAP(ix), ls='none', marker='o') for c, ix in classes.items()], 
          [c for c in classes.keys()],
          bbox_to_anchor=(1.05, 1.0), 
          loc='upper left', 
          frameon=False, 
          fontsize=12);

You can also change the number of components, for example to generate 3D figures:

In [ ]:
threeDreducer = umap.UMAP(n_components=3, n_neighbors=100, min_dist=0.01)
threeDembedding = threeDreducer.fit_transform(X)

In [ ]:
from pyrolite.util.skl.vis import alphas_from_multiclass_prob

c = DEFAULT_DISC_COLORMAP(gs.predict(X)) # predicted classes
ps = gs.predict_proba(X) # probabilities
c[:, -1] = alphas_from_multiclass_prob(ps, alpha=0.95) # append alphas to RGBA colours

In [ ]:
import ipyvolume as ipv

fig = ipv.figure()
fig = ipv.quickscatter(*threeDembedding.T,
                 color=c,
                 size=2, 
                 marker='sphere')
ipv.show()

### Using Manifold Methods for Visualizing Covariate Shift

In [ ]:
# projecting new data into manifold space

------

### Index

| [**Abstract**](./00_overview.ipynb) | **Introduction**                                                    | [**Examples**](./00_overview.ipynb#Examples)            | [**Tools**](./00_overview.ipynb#Tools) | [**Insights**](./00_overview.ipynb#Insights) |
|:-----|:-----|:-----|:-----|:-----|
|  | [Minerals Exploration](./00_overview.ipynb#An-Evolving-Role-of-Geochemistry-in-Mineral-Exploration)  | [Classification](./011_classification.ipynb) |  |  |
|  | [Data Driven Geochem](./00_overview.ipynb#Data-Driven-Geochemistry) | [Data Visualization](./012_datavis.ipynb) |  |  |